In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import numpy as np
import requests
from pathlib import Path
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans


import matplotlib.cm as cm
import matplotlib.colors as colors

import folium

In [2]:

broth = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = bs(broth)

In [3]:
table = soup.find('table',{'class':'wikitable sortable'})#Pull data from just the table
cols = table.findAll('td')

In [4]:

dataRaw = []
df = pd.DataFrame()
for i in range(len(cols)):
    dataRaw.append(cols[i].text.strip())

In [5]:
col1 = []
col2 = []
col3 = []
j = 0
for i in range(len(cols)):
    if j == 3:
        j = 0
        col1.append(dataRaw[i])
    elif j == 0:
        col1.append(dataRaw[i])
    elif j == 1: 
        col2.append(dataRaw[i])
    elif j == 2:
        col3.append(dataRaw[i])
    else:
        raise Exception('You done messed up the looping') 
    j += 1
df = pd.DataFrame()
df['Post Code'] = col1
df['Borough'] = col2
df['Neighborhood'] = col3
df.head()

,Post Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:

oldCount = df.shape
notAssignedMask = df['Borough'] == 'Not assigned'
keepMask = ~notAssignedMask
df = df[keepMask]
df.shape, oldCount

((211, 3), (288, 3))

In [7]:
df.groupby('Post Code').nunique()

,Post Code,Borough,Neighborhood
Post Code,,,
M1B,1,1,2
M1C,1,1,3
M1E,1,1,3
M1G,1,1,1
M1H,1,1,1
M1J,1,1,1
M1K,1,1,3
M1L,1,1,3
M1M,1,1,3


In [8]:

test = df.copy()
new = test.groupby('Post Code').agg({'Borough' :'first',
                                     'Neighborhood': ', '.join,
                                     }).reset_index()
df = new.copy()
df.shape

(103, 3)

In [10]:
latLong = pd.read_csv('http://cocl.us/Geospatial_data')
latLong = latLong.astype(object)
new = pd.concat([df, latLong], axis = 1, join = 'inner')
df = new.copy()

In [11]:
df.drop(['Postal Code'], axis = 1, inplace = True)

In [12]:

df = df[df['Neighborhood'].str.contains('Toronto')] 
df

,Post Code,Borough,Neighborhood,Latitude,Longitude
30,M3K,North York,"CFB Toronto, Downsview East",43.7375,-79.4648
40,M4J,East York,East Toronto,43.6853,-79.3381
46,M4R,Central Toronto,North Toronto West,43.7154,-79.4057
59,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.6408,-79.3818
60,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.6472,-79.3816
66,M5S,Downtown Toronto,"Harbord, University of Toronto",43.6627,-79.4
88,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto",43.6056,-79.5013


In [13]:
latitude = 43.6532
longitude = -79.3832

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)


for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [14]:

CLIENT_ID = 'EA42K1JKMYY34TP1YYSVLC5DXQD0IOFRRY3MQGEAXZ44VVIT'
CLIENT_SECRET = 'YGZ31K5ZFRLXOYA1B3ADUH5ZYLPNW2S3SBZ3ZMD05HCRQ2TM' 
VERSION = '20180605'

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500): #This function was built by the Coursera lab
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
LIMIT = 500
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

CFB Toronto, Downsview East
East Toronto
North Toronto West
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Harbord, University of Toronto
Humber Bay Shores, Mimico South, New Toronto


In [17]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 100 uniques categories.


In [18]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")


toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 


fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.shape

(277, 100)

In [19]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,American Restaurant,Aquarium,Art Gallery,Asian Restaurant,Bagel Shop,Bakery,Bar,...,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar
0,"CFB Toronto, Downsview East",0.000000,0.5,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00
1,"Design Exchange, Toronto Dominion Centre",0.000000,0.0,0.030000,0.00,0.01,0.01,0.000000,0.030000,0.030000,...,0.02,0.00,0.000000,0.01,0.01,0.010000,0.01,0.01,0.000000,0.01
2,East Toronto,0.000000,0.0,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00
3,"Harbord, University of Toronto",0.000000,0.0,0.000000,0.00,0.00,0.00,0.000000,0.057143,0.057143,...,0.00,0.00,0.028571,0.00,0.00,0.057143,0.00,0.00,0.028571,0.00
4,"Harbourfront East, Toronto Islands, Union Station",0.000000,0.0,0.000000,0.05,0.01,0.00,0.000000,0.020000,0.020000,...,0.01,0.01,0.010000,0.01,0.00,0.010000,0.01,0.01,0.000000,0.01
5,"Humber Bay Shores, Mimico South, New Toronto",0.000000,0.0,0.076923,0.00,0.00,0.00,0.000000,0.076923,0.000000,...,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00
6,North Toronto West,0.045455,0.0,0.000000,0.00,0.00,0.00,0.045455,0.000000,0.000000,...,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00


In [20]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----CFB Toronto, Downsview East----
                     venue  freq
0                  Airport   0.5
1                     Park   0.5
2  New American Restaurant   0.0
3                      Pub   0.0
4            Poutine Place   0.0


----Design Exchange, Toronto Dominion Centre----
         venue  freq
0  Coffee Shop  0.12
1         Café  0.08
2        Hotel  0.06
3   Restaurant  0.05
4          Bar  0.03


----East Toronto----
               venue  freq
0               Park   0.4
1       Intersection   0.2
2        Coffee Shop   0.2
3  Convenience Store   0.2
4       Noodle House   0.0


----Harbord, University of Toronto----
                 venue  freq
0                 Café  0.14
1       Sandwich Place  0.06
2  Japanese Restaurant  0.06
3              Theater  0.06
4               Bakery  0.06


----Harbourfront East, Toronto Islands, Union Station----
                 venue  freq
0          Coffee Shop  0.13
1                Hotel  0.05
2             Aquarium  0.05
3            

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']


columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.shape

(7, 11)

In [23]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)


kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)


kmeans.labels_[0:103]

array([2, 0, 1, 3, 0, 4, 0])

In [25]:

toronto_merged = df


toronto_merged['Cluster Labels'] = kmeans.labels_


toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Post Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,M3K,North York,"CFB Toronto, Downsview East",43.7375,-79.4648,2,Airport,Park,Wine Bar,Festival,Concert Hall,Convenience Store,Dance Studio,Deli / Bodega,Dessert Shop,Diner
40,M4J,East York,East Toronto,43.6853,-79.3381,0,Park,Coffee Shop,Intersection,Convenience Store,Fast Food Restaurant,Comfort Food Restaurant,Concert Hall,Dance Studio,Deli / Bodega,Dessert Shop
46,M4R,Central Toronto,North Toronto West,43.7154,-79.4057,1,Coffee Shop,Clothing Store,Sporting Goods Shop,Yoga Studio,Salon / Barbershop,Health & Beauty Service,Italian Restaurant,Mexican Restaurant,Diner,Dessert Shop
59,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.6408,-79.3818,3,Coffee Shop,Aquarium,Hotel,Café,Brewery,Fried Chicken Joint,Scenic Lookout,Plaza,Pizza Place,History Museum
60,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.6472,-79.3816,0,Coffee Shop,Café,Hotel,Restaurant,Gastropub,American Restaurant,Bakery,Bar,Italian Restaurant,Deli / Bodega


In [26]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clustersmap_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters